# **ДОМАШНЕЕ ЗАДАНИЕ 1**



## **1) Загрузить файл с данными по клиентам и транзакциям.** 👌  
___


## **2) Продумать структуру базы данных и отрисовать в редакторе.** 👌  
___
Дальше по алгоритму идут шаги по нормализации, поэтому сначала я отрисовала структуру, как в excel-файле из задания.
![структура_1.png](скрины/структура_1.png)
Код из редактора:
```sql
Table Transaction {
  transaction_id integer
  product_id	integer
  customer_id	integer
  transaction_date	timestamp
  online_order	BOOLEAN
  order_status	varchar
  brand	varchar
  product_line	varchar
  product_class	varchar
  product_size	varchar
  list_price	Float
  standard_cost Float
}
Table Customers {
  customer_id integer
  first_name varchar
  last_name varchar
  gender varchar
  DOB date
  job_title varchar
  job_industry_category varchar
  wealth_segment varchar
  deceased_indicator varchar
  owns_car varchar
  address varchar
  postcode varchar
  state varchar
  country varchar
  property_valuation integer
}
Ref: "Transaction"."customer_id" < "Customers"."customer_id"
```

## **3) Нормализовать базу данных (от 1НФ до 3НФ), описав, к какой нормальной форме приводится таблица и почему таблица в этой нормальной форме изначально не находилась.** 👌  
___
  
### 1НФ. Для того чтобы таблица была в 1НФ все атрибуты в таблице должны быть простыми, все сохраняемые данные на пересечении столбцов и строк — содержать лишь скалярные значения, а строки не должны повторяться. В структуре в п.2 нет гарантии, что строки не будут повторяться. Поэтому надо задать primary key:
```sql
...
transaction_id integer [primary key]
...
customer_id integer [primary key]
...
```
![Структура_2.png](скрины/Структура_2.png)
   
### 2НФ. Отношения будут соответствовать 2НФ, если сама БД находится в 1НФ, а каждый столбец, который не является ключом, зависит от первичного ключа. В структуре не все строки зависят от первичного ключа, например, в таблице customer поле "postcode" зависит от поля "address".
![Структура_3.png](скрины/Структура_3.png)
```sql
Table Transaction {
  transaction_id integer [primary key]
  product_id	integer
  customer_id	integer
  transaction_date	timestamp
  online_order	BOOLEAN
  order_status	varchar
}
Table Product {
  product_id	integer [primary key]
  product_class	varchar 
  product_size	varchar 
  brand	varchar
  product_line	varchar 
  list_price float
  standard_cost float
}
Table Customers {
  customer_id integer [primary key]
  first_name varchar
  last_name varchar
  gender varchar
  DOB date
  job_title varchar
  job_industry_category varchar
  wealth_segment varchar
  deceased_indicator varchar
  owns_car varchar
  address_id integer
}
Table Addresses {
  address_id integer [primary key]
  address_line varchar
  postcode varchar
  state varchar
  country varchar
  property_valuation integer
}
Ref: "Transaction"."product_id" < "Product"."product_id"
Ref: "Transaction"."customer_id" < "Customers"."customer_id"
Ref: "Customers"."address_id" < "Addresses"."address_id"
```
### 3) 3НФ. Для 3НФ нужно чтобы, все колонки в таблице зависели от первичного ключа и не зависели друг от друга. В структуре выше есть зависимые друг от друга поля: например, поля postcode и state зависят. Исправленная структура:
![Структура_4.png](скрины/Структура_4.png)
``` sql
Table Transaction {
    transaction_id integer [primary key]
    product_id integer
    customer_id integer
    transaction_date timestamp
    online_order boolean
    order_status varchar
  }
  Table Product {
    product_id integer [primary key]
    brand varchar
    product_line varchar
    product_class varchar
    product_size varchar
    list_price float
    standard_cost float
  }
  Table Customers {
    customer_id integer [primary key]
    first_name varchar
    last_name varchar
    gender varchar
    DOB date
    job_id integer
    deceased_indicator varchar
    owns_car varchar
    address_id integer
    wealth_segment varchar
  }
  Table Job {
    job_id integer [primary key]
    job_title varchar
    job_industry_category varchar
  }
  Table Postcode {
    postcode_id integer [primary key]
    postcode varchar
    state varchar
    country varchar
  }
  Table Addresses {
    address_id integer [primary key]
    address_line varchar
    postcode_id varchar 
    property_valuation integer
  }
  Ref: "Transaction"."product_id" < "Product"."product_id"
  Ref: "Transaction"."customer_id" < "Customers"."customer_id"
  Ref: "Customers"."address_id" < "Addresses"."address_id"
  Ref: "Addresses"."postcode_id" < "Postcode"."postcode_id"
  Ref: "Customers"."job_id" < "Job"."job_id"
```

## **4) Создать все таблицы в DBeaver, указав первичные ключи к таблицам, правильные типы данных, могут ли поля быть пустыми или нет.** 👌  
___

In [1]:
#!pip install psycopg2

In [2]:
import psycopg2
db_config = {
    'host': 'localhost',
    'database': 'hw1',
    'user': 'postgres',
    'password': '984651',
    'port': '5432'
}
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

In [3]:
conn.rollback()

In [4]:
cursor.execute("""
        DROP TABLE IF EXISTS Transaction;
        DROP TABLE IF EXISTS Customers;
        DROP TABLE IF EXISTS Product;
        DROP TABLE IF EXISTS Addresses;
        DROP TABLE IF EXISTS Postcode;
        DROP TABLE IF EXISTS Job;
    """)

cursor.execute("""
        CREATE TABLE Postcode (
            postcode_id INTEGER PRIMARY KEY,
            postcode VARCHAR NOT NULL,
            state VARCHAR NOT NULL,
            country VARCHAR NOT NULL
        );
        
        CREATE TABLE Job (
            job_id INTEGER PRIMARY KEY,
            job_title VARCHAR,
            job_industry_category VARCHAR NOT NULL
        );
        
        CREATE TABLE Addresses (
            address_id INTEGER PRIMARY KEY,
            address_line VARCHAR NOT NULL,
            postcode_id INTEGER NOT NULL,
            property_valuation INTEGER NOT NULL,
            FOREIGN KEY (postcode_id) REFERENCES Postcode(postcode_id)
        );
        
        CREATE TABLE Product (
            product_id INTEGER PRIMARY KEY,
            brand VARCHAR,
            product_line VARCHAR,
            product_class VARCHAR,
            product_size VARCHAR,
            list_price DECIMAL(10,2) NOT NULL,
            standard_cost DECIMAL(10,2)
        );
        
        CREATE TABLE Customers (
            customer_id INTEGER PRIMARY KEY,
            first_name VARCHAR NOT NULL,
            last_name VARCHAR,
            gender VARCHAR NOT NULL,
            DOB DATE,
            job_id INTEGER NOT NULL,
            deceased_indicator VARCHAR NOT NULL,
            owns_car VARCHAR NOT NULL,
            address_id INTEGER NOT NULL,
            wealth_segment VARCHAR NOT NULL,
            FOREIGN KEY (job_id) REFERENCES Job(job_id),
            FOREIGN KEY (address_id) REFERENCES Addresses(address_id)
        );
        
        CREATE TABLE Transaction (
            transaction_id INTEGER PRIMARY KEY,
            product_id INTEGER NOT NULL,
            customer_id INTEGER NOT NULL,
            transaction_date TIMESTAMP NOT NULL,
            online_order BOOLEAN,
            order_status VARCHAR NOT NULL,
            FOREIGN KEY (product_id) REFERENCES Product(product_id),
            FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
        );
""")
conn.commit()
cursor.close()
conn.close()

### Скрин с таблицами из DBeaver
![Схема_DBeaver.png](скрины/Схема_DBeaver.png)

## **5) Загрузить данные в таблицы в соответствии с созданной структурой** 👌
___

Загрузила в DBeaver через import scv
  
### job
![job_import.png](скрины/job_import.png)
  
### postcode
![postcode_import.png](скрины/postcode_import.png)
  
### product
![product_import.png](скрины/product_import.png)
  
### addresses
![addresses_import.png](скрины/addresses_import.png)
  
### customers
![customers_import.png](скрины/customers_import.png)
  
### transaction
![transaction_import.png](скрины/transaction_import.png)